In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from collections import Counter
import openai
import ast


def deduplicate_csv(csv_path, columns, similarity_threshold=0.80):
    df = pd.read_csv(csv_path)

    df['combined_text'] = df[columns].fillna('').agg(' '.join, axis=1)

    # Create the TF-IDF matrix
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['combined_text'])

    # Calculate the cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix)

    # Find duplicates: records with similarity above the threshold
    duplicates_indices = set()
    for i in range(cosine_sim.shape[0]):
        for j in range(i+1, cosine_sim.shape[0]):
            if cosine_sim[i, j] > similarity_threshold:
                # Choose one of the duplicates to drop, e.g., the one with the higher index
                duplicates_indices.add(max(i, j))

    # Drop the duplicates and reset the index
    df_deduplicated = df.drop(index=duplicates_indices).reset_index(drop=True)

    return df_deduplicated

# Specify the path to your CSV file and which columns to consider for deduplication
csv_file_path = 'projects_output_diabetes_combined.csv'
columns_to_consider = ['title', 'main_goal_and_aims']

# Load the dataset
df = pd.read_csv(csv_file_path)
print(f"Number of records before deduplication: {df.shape[0]}")

# Perform deduplication
df_deduplicated = deduplicate_csv(csv_file_path, columns_to_consider)
print(f"Number of records after deduplication: {df_deduplicated.shape[0]}")

# Save the deduplicated dataframe to a new CSV file
df_deduplicated.to_csv('deduplicated_projects_output.csv', index=False)


Number of records before deduplication: 573
Number of records after deduplication: 407


In [ ]:
df = pd.read_csv(csv_file_path)
print(df.columns.tolist())

['title', 'main_goal_and_aims', 'Questions', 'Purpose', 'Approaches', 'Findings', 'Categories', 'DataTier', 'Team']


In [ ]:


def extract_team_info(team_text):
    team_members = [member.strip() for member in team_text.split('-') if member.strip()]
    individuals = []
    institutions = []
    for member in team_members:
        parts = member.rsplit(',', 1)
        name = parts[0].strip()
        institution = parts[1].strip() if len(parts) > 1 else None
        individuals.append(name)
        if institution:
            institutions.append(institution)
    return individuals, institutions

def calculate_team_metrics(df):
    all_individuals = []
    all_institutions = []
    multi_institutional_count = 0

    for team_text in df['Team']:
        individuals, institutions = extract_team_info(team_text)
        all_individuals.extend(individuals)
        all_institutions.extend(institutions)
        if len(set(institutions)) > 1:
            multi_institutional_count += 1

    unique_individuals = len(set(all_individuals))
    unique_institutions = len(set(all_institutions))
    avg_individuals_per_team = len(all_individuals) / len(df)
    multi_institutional_percentage = (multi_institutional_count / len(df)) * 100

    return unique_individuals, unique_institutions, avg_individuals_per_team, multi_institutional_percentage

def calculate_category_metrics(df):
    demographic_category_count = 0
    for categories in df['Categories']:
        if "This study will not center on underrepresented populations." not in categories:
            demographic_category_count += 1
    percentage_with_demographic = (demographic_category_count / len(df)) * 100
    return percentage_with_demographic

# Load the deduplicated data
df = pd.read_csv('deduplicated_projects_output.csv')  # Replace with the actual path to the deduplicated CSV

# Calculate metrics related to the team
unique_individuals, unique_institutions, avg_individuals_per_team, multi_institutional_percentage = calculate_team_metrics(df)

# Calculate metrics related to the categories
percentage_with_demographic = calculate_category_metrics(df)

# Print the metrics
print(f"# of registered projects (deduplicated): {len(df)}")
#print(f"# of unique individuals listed as team-members: {unique_individuals}")
#print(f"Average number of individuals per team: {avg_individuals_per_team:.2f}")
print(f"% projects using demographic categories for study: {percentage_with_demographic:.2f}%")
#print(f"# of unique institutions: {unique_institutions}")
#print(f"% projects with multi-institutional teams: {multi_institutional_percentage:.2f}%")


# of registered projects (deduplicated): 407
% projects using demographic categories for study: 51.84%


In [ ]:


df = pd.read_csv('deduplicated_projects_output.csv')


sample_df = df

openai.api_key = ''
model = "gpt-3.5-turbo-0125"

results = []
for index, row in sample_df.iterrows():
    prompt = (
        'Given the team information, I want you to extract individual names, roles, and institutions '
        'and return the information in three separate Python lists.\n\n'
        'Example 1:\n'
        'Team information: \"Research Team Owner: Elizabeth Theusch - Other, University of California, San Francisco\"\n'
        '1. ["Owner"]\n'
        '2. ["Elizabeth Theusch"]\n'
        '3. ["University of California, San Francisco"]\n\n'
        'Example 2:\n'
        'Team information: \"Research Team Owner: Xinyi Tan - Graduate Trainee, Columbia University '
        'Jay Lusk - Research Fellow, Duke University\"\n'
        '1. ["Owner", "Graduate Trainee", "Research Fellow"]\n'
        '2. ["Xinyi Tan", "Jay Lusk"]\n'
        '3. ["Columbia University", "Duke University"]\n\n'
        'Now, extract the individual names, roles, and institutions from the following team information as three Python lists:\n'
        f"{row['Team']}\n"
    )
    messages = [{"role": "system", "content": "Extract the individual names, roles, and institutions. Follow the format in the prompt strictly"},
                {"role": "user", "content": prompt }]
    try:
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        extracted_info = response.choices[0].message.content
        results.append(extracted_info)
        #print(f"Original: {row['Team']}\nExtracted: {extracted_info}\n")
        #print(f"Extracted: {extracted_info}\n")
    except openai.OpenAIError as e:
        print(f"An error occurred: {e}")
        results.append(None)




sample_df['Extracted Team Info'] = results
sample_df.to_csv('extracted_team_info_sample.csv', index=False)


In [ ]:
def clean_and_parse(info):
    info = (info.replace('Roles:', '')
               .replace('Names:', '')
               .replace('Institutions:', '')
               .replace('1. ', '')
               .replace('2. ', '')
               .replace('3. ', '')
               .replace('Roles List:','')
               .replace('Names List:','')
               .replace('Institutions List:','')
               )

    print(info)
    try:

        parts = [part.strip() for part in info.split('\n') if part.strip()]

        roles = ast.literal_eval(parts[0])
        researchers = ast.literal_eval(parts[1])
        institutions = ast.literal_eval(parts[2])
        return roles, researchers, institutions
    except Exception as e:
        print(f"An error occurred while parsing: {e}")
        return [], [], []


extracted_team = pd.read_csv('extracted_team_info_sample.csv')

parsed_results = []
for index, row in extracted_team.iterrows():
    roles, researchers, institutions = clean_and_parse(row['Extracted Team Info'])
    parsed_results.append({
        'Roles': roles,
        'Researchers': researchers,
        'Institutions': institutions
    })


df_parsed = pd.DataFrame(parsed_results)


df_parsed.to_csv('parsed_results.csv', index=False)


df_parsed.head()



["Owner", "Research Fellow"]
["Sarada Ghosh"]
["Cornell University"]
["Owner", "Graduate Trainee", "Research Fellow"]
["Xinyi Tan", "Jay Lusk"]
["Columbia University", "Duke University"]
["Owner", "Collaborator"]
["Reagan Ballard", "Micah Hysong"]
["University of North Carolina, Chapel Hill", "University of North Carolina, Chapel Hill"]
 ["Robin Beaumont", "Michael Weedon", "Gareth Hawkes", "Andrew Wood", "Harry Wright", "Harry Green"]
 ["Research Fellow", "Senior Researcher", "Research Fellow", "Mid-career Tenured Researcher", "Research Assistant", "Mid-career Tenured Researcher"]
 ["University of Exeter", "University of Exeter", "University of Exeter", "University of Exeter", "University of Exeter", "University of Exeter"]
["Owner", "Project Personnel"]
["Yuji Choi"]
["Johns Hopkins University"]
["Owner", "Collaborator", "Collaborator"]
["Jonathon Baccus", "Hui Wang", "Graham Jones"]
["University of Alabama", "University of Alabama", "University of Alabama"]
["Owner", "Graduate Train

,Roles,Researchers,Institutions
0,"[Owner, Research Fellow]",[Sarada Ghosh],[Cornell University]
1,"[Owner, Graduate Trainee, Research Fellow]","[Xinyi Tan, Jay Lusk]","[Columbia University, Duke University]"
2,"[Owner, Collaborator]","[Reagan Ballard, Micah Hysong]","[University of North Carolina, Chapel Hill, Un..."
3,"[Robin Beaumont, Michael Weedon, Gareth Hawkes...","[Research Fellow, Senior Researcher, Research ...","[University of Exeter, University of Exeter, U..."
4,"[Owner, Project Personnel]",[Yuji Choi],[Johns Hopkins University]


In [ ]:

unique_individuals = set()
unique_institutions = set()
total_individual_count = 0
multi_institutional_projects_count = 0


for result in parsed_results:
    researchers = result['Researchers']
    institutions = result['Institutions']


    unique_individuals.update(researchers)
    unique_institutions.update(institutions)
    total_individual_count += len(researchers)


    if len(set(institutions)) > 1:
        multi_institutional_projects_count += 1


average_individuals_per_team = total_individual_count / len(parsed_results)
percent_multi_institutional_teams = (multi_institutional_projects_count / len(parsed_results)) * 100

print(f"Average number of individuals per team: {average_individuals_per_team:.2f}")
print(f"# of unique individuals listed as team-members: {len(unique_individuals)}")
print(f"# of unique institutions: {len(unique_institutions)}")
print(f"% projects with multi-institutional teams: {percent_multi_institutional_teams:.2f}%")


Average number of individuals per team: 1.91
# of unique individuals listed as team-members: 600
# of unique institutions: 176
% projects with multi-institutional teams: 15.48%


In [ ]:

df_parsed = pd.read_csv('parsed_results.csv', converters={
    'Researchers': ast.literal_eval,
    'Institutions': ast.literal_eval
})


df_parsed['Individuals Per Team'] = df_parsed['Researchers'].apply(len)
df_parsed['Unique Individuals Per Entry'] = df_parsed['Researchers'].apply(lambda x: len(set(x)))
df_parsed['Unique Institutions Per Entry'] = df_parsed['Institutions'].apply(lambda x: len(set(x)))
df_parsed['Is Multi-Institutional'] = df_parsed['Institutions'].apply(lambda x: len(set(x)) > 1)
df_parsed.to_csv('qa_metrics_per_entry.csv', index=False)


In [ ]:

df_extracted = pd.read_csv('extracted_team_info_sample.csv')
df_parsed = pd.read_csv('parsed_results.csv')
df_qa_metrics = pd.read_csv('qa_metrics_per_entry.csv')


df_combined = pd.concat([df_extracted, df_parsed, df_qa_metrics], axis=1)


df_combined.to_csv('combined_data.csv', index=False)
